In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

sentence1 = "translate English to French:"
sentence2 = "apple is so good"

print(tokenizer(sentence1, sentence2,
    padding="longest",

    return_tensors="pt",
))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


{'input_ids': tensor([[13959,  1566,    12,  2379,    10,     1,  8947,    19,    78,   207,
             1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [14]:
class LINEAR(nn.Module):
    def __init__(self, num):
        super().__init__()

        self.module = self.setup_linear(num)

    def setup_linear(self, num):
        _list = []
        for _ in range(num):
            _linear = nn.Linear(10, 5)
            _list.append(_linear)

        return nn.ModuleList(_list)
    
    def forward(self, x):

        return x
    
l_list = []
num = 5
for _ in range(num):
    l_list.append(LINEAR(num).setup_linear(num))

    

Parameter containing:
tensor([[ 0.0261,  0.0434, -0.1289, -0.0158,  0.2814, -0.2877, -0.1675, -0.3108,
          0.2470,  0.1983],
        [-0.0460,  0.1326,  0.2456, -0.2517, -0.2937,  0.2213,  0.0970, -0.2704,
         -0.0220, -0.1705],
        [ 0.2078, -0.2895, -0.1389, -0.0434, -0.1690,  0.0375,  0.1244, -0.1545,
         -0.0084,  0.1417],
        [-0.0710, -0.0569, -0.1636,  0.2336, -0.2478, -0.1569, -0.0549, -0.0583,
          0.3083, -0.2320],
        [-0.2403, -0.2796, -0.2459, -0.1995,  0.1914, -0.2359, -0.1594,  0.2871,
         -0.2707, -0.1412]], requires_grad=True)
Parameter containing:
tensor([[-0.2308,  0.2195,  0.1126, -0.0944, -0.0218, -0.2988, -0.2593,  0.1112,
          0.1572,  0.2109],
        [-0.2542, -0.1612, -0.1532,  0.1923, -0.0717, -0.0114, -0.0017, -0.1877,
          0.2724,  0.2089],
        [ 0.1491, -0.2754, -0.1470,  0.0663, -0.0885, -0.0690, -0.0230,  0.1277,
          0.2878,  0.2793],
        [-0.1641,  0.0487,  0.1721, -0.0723, -0.0646, -0.0599, 

In [52]:
import numpy as np
a = torch.ones((3,5), dtype=torch.long)

print(torch.stack([a, a],dim=0).shape)


torch.Size([2, 3, 5])


In [28]:
_list = []

for _ in range(5):
    linear_module = nn.Linear(5, 5)
    _list.append(linear_module)

linear_module_list = nn.ModuleList(_list)
linear_module_sequence = nn.Sequential(linear_module_list)

tensor = torch.ones((5, 5), dtype=torch.float)
output = linear_module_sequence(tensor)

class k(nn.Module):
    def __init__(self):
        super().__init__()

    def _setup(self):
        _list = []
        for _ in range(5):
            linear_module = nn.Linear(5, 5)
            _list.append(linear_module)

        linear_module_list = nn.ModuleList(_list)
        self.linear_module_sequence = nn.Sequential(linear_module_list)

    def forward(self, input):
        output = self.linear_module_sequence(input)

        return output

model = k()

print(k(tensor))


NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [19]:
import torch
import torch.nn as nn

class A:
    def __init__(self):
        pass


class B(A):
    def __init__(self, name):
        super().__init__()
        self.linear = nn.Linear(10, 10)


class C(A):
    def __init__(self, name):
        super().__init__()
        self.linear = nn.Linear(10, 10)


_list = []
for _ in range(3):
    b= B("name")
    c= C("name")
    _list.append(b)
    _list.append(c)



In [ ]:
kwargs = {
    "batch_size":32,
    "seq_length":64,
    "hidden_size":768,
    "num_layer":12,
    "num_heads":12,
    "ffn_hidden_size":3072,
    "num_classes":100
}

class T5(nn.Module):
    def __init__(self, batch_size, seq_length, hidden_size, num_layer, num_heads, ffn_hidden_size, num_classes):
        super().__init__()
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.num_heads = num_heads
        self.ffn_hidden_size = ffn_hidden_size

        self.encoder = Encoder()
        self.decoder = Decoder()
        self.encoder_embedding = nn.Embedding(seq_length + 1, hidden_size)
        self.decoder_embedding = nn.Embedding(seq_length + 1, hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, encoder_input_ids, decoder_input_ids, encoder_attention_mask, decoder_attention_mask):
        encoder_embedding = self.encoder_embedding(encoder_input_ids)
        output_encoder = self.encoder(encoder_embedding, encoder_attention_mask)
        decoder_embedding = decoder_embedding(decoder_input_ids)
        decoder_output = self.decoder(decoder_embedding, output_encoder, encoder_attention_mask, decoder_attention_mask)
        logits = self.fc(decoder_output)

        return logits

class Encoder(T5):
    def __init__(self):
        super().__init__()
        self._setupEncoderLayer(self.num_layer)

    def _setupEncoderLayer(self, num_layer):
        encoder_layer_list = []
        for _ in range(num_layer):
            encoder_layer = EncoderLayer()
            encoder_layer_list.append(encoder_layer)
        self.encoder_module = nn.ModuleList(encoder_layer_list)

    def forward(self, encoder_embedding, encoder_attention_mask):
        tokens = encoder_embedding
        for encoder_layer in self.encoder_module:
            tokens = encoder_layer(tokens, encoder_attention_mask)
        output_encoder = tokens

        return output_encoder
    
class EncoderLayer(Encoder):
    def __init__(self):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(self.batch_size, self.num_heads, self.seq_length, self.hidden_size, check_positional_embedding=True, check_mask=False)
        self.add_norm1 = AddNorm(self.batch_size, self.seq_length, self.hidden_size)
        self.feed_forward = FeedForward(self.hidden_size, self.ffn_hidden_size)
        self.add_norm2 = AddNorm(self.batch_size, self.seq_length, self.hidden_size)

    def forward(self, tokens, encoder_attention_mask):
        skip1 = tokens
        multi_head_attention = self.multi_head_attention(tokens, tokens, tokens, encoder_attention_mask, encoder_attention_mask)
        add_norm1 = self.add_norm1(multi_head_attention, skip1)
        skip2 = add_norm1
        feed_forward = self.feed_forward(add_norm1)
        add_norm2 = self.add_norm2(feed_forward, skip2)
        tokens = add_norm2

        return tokens
    
    
class Decoder(T5):
    def __init__(self):
        super().__init__()
        self._setupDecoderLayer(self.num_layer)

    def _setupDecoderLayer(self, num_layer):
        decoder_list = []
        for _ in range(num_layer):
            decoder_layer = DecoderLayer()
            decoder_list.append(decoder_layer)
        self.decoder_module = nn.ModuleList(decoder_list)

    def forward(self, decoder_embedding, output_encoder, encoder_attention_mask, decoder_attention_mask):
        tokens = decoder_embedding
        for decoder_layer in self.decoder_module:
            tokens = decoder_layer(tokens, output_encoder, encoder_attention_mask, decoder_attention_mask)
        output_decoder = tokens

        return output_decoder
    
class DecoderLayer(Decoder):
    def __init__(self):
        super().__init__()
        self.masked_multi_head_attention =  MultiHeadAttention(self.batch_size, self.num_heads, self.seq_length, self.hidden_size, check_positional_embedding=True, check_mask=True)
        self.add_norm1 = AddNorm(self.batch_size, self.seq_length, self.hidden_size)
        self.cross_multi_head_attention = MultiHeadAttention(self.batch_size, self.num_heads, self.seq_length, self.hidden_size, check_positional_embedding=False, check_mask=False)
        self.add_norm2 = AddNorm(self.batch_size, self.seq_length, self.hidden_size)
        self.feed_forward = FeedForward(self.hidden_size, self.ffn_hidden_size)
        self.add_norm3 = AddNorm(self.batch_size, self.seq_length, self.hidden_size)

    def forward(self, tokens, output_encoder, encoder_attention_mask, decoder_attention_mask):
        skip1 = tokens
        masked_multi_head_attention = self.masked_multi_head_attention(tokens, tokens, tokens, decoder_attention_mask, decoder_attention_mask)
        add_norm1 = self.add_norm1(masked_multi_head_attention, skip1)
        skip2 = add_norm1
        cross_multi_head_attention = self.cross_multi_head_attention(tokens, output_encoder, output_encoder, decoder_attention_mask, encoder_attention_mask)
        add_norm2 = self.add_norm2(cross_multi_head_attention, skip2)
        skip3 = add_norm2
        feed_forward = self.feed_forward(tokens)
        add_norm3 = self.add_norm3(feed_forward, skip3)
        tokens = add_norm3

        return tokens
    
class MultiHeadAttention(T5):
    def __init__(self):
        super().__init__()

    def _setupHeadQKV(self, num_heads, hidden_size):
        query_module = []
        key_module = []
        value_module = []
        head_hidden_size = int(hidden_size / num_heads)

        for _ in range(num_heads):
            query_module.append(nn.Linear(hidden_size, head_hidden_size))
            key_module.append(nn.Linear(hidden_size, head_hidden_size))
            value_module.append(nn.Linear(hidden_size, head_hidden_size))

        self.query_module = nn.ModuleList(query_module)
        self.key_module = nn.ModuleList(key_module)
        self.value_module = nn.ModuleList(value_module)

    def _outputRelativePositionalEmbeddingScalar(self, query, batch_size, seq_length, hidden_size, num_heads):
        embed_Module = []
        head_hidden_size = int(hidden_size / num_heads)
        position_ids = torch.tensor(list(range(seq_length)), dtype=torch.long).reshape(1, seq_length).expand(batch_size, seq_length)
        for id in range(num_heads): embed_Module.append(nn.Embedding(seq_length + 1, head_hidden_size))
        self.embed_module = nn.ModuleList(embed_Module)
        for id in range(num_heads):
            head_query = self.query_module[id](query)
            tmp_relative_position_embedding_scalar = head_query@(self.embed_module[id](position_ids).transpose(1, 2))
            if id == 0: relative_position_embedding_scalar = tmp_relative_position_embedding_scalar
            else: trelative_position_embedding_scalar = torch.concat([trelative_position_embedding_scalar, tmp_relative_position_embedding_scalar], dim=0)

        return relative_position_embedding_scalar
    
    def _outputAttention(self, query, key, value, batch_size, seq_length, hidden_size, num_heads, check_positional_embedding, check_mask, encoder_attention_mask, decoder_attention_mask):
        head_hidden_size = int(hidden_size / num_heads)
        encoder_attention_mask = encoder_attention_mask.reshape(batch_size, -1, 1).expand(batch_size, seq_length, seq_length)
        decoder_attention_mask = decoder_attention_mask.reshape(batch_size, 1, -1).expand(batch_size, seq_length, seq_length)
        padding_map = encoder_attention_mask * decoder_attention_mask
        mask_map = torch.tensor(np.tril(np.ones((seq_length, seq_length))), dtype=torch.long)

        if check_positional_embedding:relative_position_embedding_scalar = self._outputRelativePositionalEmbeddingScalar(query, batch_size, seq_length, hidden_size, num_heads)
        else:relative_position_embedding_scalar = torch.tensor(0, torch.float)

        for id in range(num_heads):
            head_query = self.query_module[id](query)
            head_key = self.key_module[id](key)
            head_value = self.value_module[id](value)

            if check_mask: tmp_head_attention = nn.Softmax(padding_map * mask_map * (head_query@head_key.transpose(1, 2)) / (head_hidden_size) + relative_position_embedding_scalar[id])@head_value
            else: tmp_head_attention = nn.Softmax(padding_map * (head_query@head_key.transpose(1, 2)) / (head_hidden_size) + relative_position_embedding_scalar[id])@head_value

            if id == 0: head_attention = tmp_head_attention
            else: head_attention = torch.concat([head_attention, tmp_head_attention], dim=-1)

        output_attention = head_attention

        return output_attention

    def forward(self, query, key, value, encoder_attention_mask, decoder_attention_mask):
        output_attention = self._outputAttention(self, query, key, value, self.batch_size, self.seq_length, self.hidden_size, self.num_heads,
                                                  self.check_positional_embedding, self.check_mask, encoder_attention_mask, decoder_attention_mask):

        return output_attention
    
class AddNorm(T5):
    def __init__(self):
        super().__init__()
        self._setupAddNormModule(self.batch_size, self.seq_length, self.hidden_size)

    def _setupAddNormModule(self, batch_size, seq_length, hiddne_size):
        self.layer_norm = nn.LayerNorm(batch_size, seq_length, hiddne_size)

    def forward(self, tokens, skipped_tokens):
        tokens += tokens + skipped_tokens
        tokens = self.layer_norm(tokens)

        return tokens
    
class FeedForward(T5):
    def __init__(self):
        super().__init__()
        self._setupFeedForwardModule(self.hidden_size, self.ffn_hidden_size)

    def _setupFeedForwardModule(self, hidden_size, ffn_hidden_size):
        dense1 = nn.Linear(hidden_size, ffn_hidden_size)
        relu1 = nn.ReLU()
        dense2 = nn.Linear(ffn_hidden_size, hidden_size)
        relu2 = nn.ReLU()
        self.feed_foward_module = nn.ModuleList([dense1, relu1, dense2, relu2])

    def forward(self, tokens):
        for module in self.feed_foward_module:
            tokens = module(tokens)
        return tokens
